In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from concrete.ml.sklearn import LinearRegression

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    x, y = make_classification(n_samples=100, class_sep=2, n_features=30, random_state=42)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_pred_clear = model.predict(x_test)
    model.compile(x_train)
    y_pred_fhe = model.predict(x_test, fhe="execute")
    print(y_pred_clear)
    print(y_pred_fhe)
    normalized = StandardScaler().fit_transform(y_pred_fhe) * 100

    return {'result': normalized.tolist()}

@app.get('/get_score/{uid}')
async def get_score(uid: str):
    x, y = make_classification(n_samples=100, class_sep=2, n_features=30, random_state=42)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_pred_clear = model.predict(x_test)
    model.compile(x_train)
    y_pred_fhe = model.predict(x_test, fhe="execute")
    print(y_pred_clear)
    print(y_pred_fhe)
    normalized = StandardScaler().fit_transform(y_pred_fhe)

    score = normalized.tolist()[int(uid) % len(normalized.tolist())][0] * 100
    return {'score': int(score)}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)